# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [ ]:
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="append")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

Pipeline dlt_colab_kernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707718469.3896847 is LOADED and contains no failed jobs


In [ ]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

rides = conn.sql("SELECT * FROM people").df()
display(rides)

age = conn.sql("SELECT SUM(Age) FROM people").df()
display(age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707718469.3896847,IdC5gzzFH/8wRA,None
1,2,Person_2,27,City_A,1707718469.3896847,m+8sZfR80oRuaw,None
2,3,Person_3,28,City_A,1707718469.3896847,8LZiAT3ghiw3rA,None
3,4,Person_4,29,City_A,1707718469.3896847,sv8T7SK02wR9hA,None
4,5,Person_5,30,City_A,1707718469.3896847,KYMCPhT9BpXNbA,None
5,3,Person_3,33,City_B,1707718470.2011116,6GP7QHoGIKV2yQ,Job_3
6,4,Person_4,34,City_B,1707718470.2011116,e3/m4JLxVQXBBw,Job_4
7,5,Person_5,35,City_B,1707718470.2011116,YFtltyCs05p0SQ,Job_5
8,6,Person_6,36,City_B,1707718470.2011116,XEThxraTDOWhuA,Job_6
9,7,Person_7,37,City_B,1707718470.2011116,5EXkd8G1z+Ourg,Job_7


,sum(Age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators_2')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_2",
										write_disposition="merge",
                    primary_key='id')

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)
info = generators_pipeline.run(people_2(),
										table_name="people_2",
										write_disposition="merge",
                    primary_key='id')

Pipeline dlt_colab_kernel_launcher load step completed in 0.41 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_2
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707720742.0378833 is LOADED and contains no failed jobs


In [ ]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

rides = conn.sql("SELECT * FROM people_2").df()
display(rides)

age = conn.sql("SELECT SUM(Age) FROM people_2").df()
display(age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_2            │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1707720742.0378833,e1q1JUM/fT6XFQ,None
1,1,Person_1,26,City_A,1707720742.0378833,Gwf7VRGk9skpOw,None
2,8,Person_8,38,City_B,1707720742.8635926,SbYwg6YPPooUVw,Job_8
3,4,Person_4,34,City_B,1707720742.8635926,iakmp9wlNgR0Yg,Job_4
4,5,Person_5,35,City_B,1707720742.8635926,P3G1gHByKXANKw,Job_5
5,7,Person_7,37,City_B,1707720742.8635926,wBVy8ABX9F7G4g,Job_7
6,3,Person_3,33,City_B,1707720742.8635926,GKUybMb9qJXmvg,Job_3
7,6,Person_6,36,City_B,1707720742.8635926,qbK4x4DfdT0iGA,Job_6


,sum(Age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX